In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import numpy as np

In [3]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data transformations with Mixup (augmentation) and normalization
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_val = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load dataset
train_dataset = datasets.ImageFolder(root="/kaggle/input/recycle-with-split/recycle_split/train", transform=transform_train)
val_dataset = datasets.ImageFolder(root="/kaggle/input/recycle-with-split/recycle_split/test", transform=transform_val)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [6]:

# Load the MobileNetV2 model pre-trained on ImageNet
num_classes = 4 
model = models.mobilenet_v2(pretrained=True)
model.classifier[1] = nn.Linear(model.last_channel, num_classes)  # Adjust for number of classes

model = model.to(device)

# Label smoothing criterion
class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes, smoothing=0.1):
        super(LabelSmoothingLoss, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.cls = classes
        self.log_softmax = nn.LogSoftmax(dim=-1)

    def forward(self, x, target):
        with torch.no_grad():
            true_dist = torch.zeros_like(x)
            true_dist.fill_(self.smoothing / (self.cls - 1))
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        return torch.mean(torch.sum(-true_dist * self.log_softmax(x), dim=-1))

num_classes = 4  # Update with your number of classes
criterion = LabelSmoothingLoss(classes=num_classes, smoothing=0.1)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [8]:
# Training the model
num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, predicted = outputs.max(1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = 100 * correct / total
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%')

# Validation loop
model.eval()
val_loss = 0.0
correct = 0
total = 0
with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        val_loss += loss.item() * images.size(0)
        _, predicted = outputs.max(1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

val_loss /= total
val_accuracy = 100 * correct / total
print(f'Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%')


Epoch [1/30], Loss: 0.7684, Accuracy: 81.53%
Epoch [2/30], Loss: 0.8053, Accuracy: 79.12%
Epoch [3/30], Loss: 0.7748, Accuracy: 80.81%
Epoch [4/30], Loss: 0.7594, Accuracy: 81.77%
Epoch [5/30], Loss: 0.7508, Accuracy: 82.50%
Epoch [6/30], Loss: 0.7493, Accuracy: 82.79%
Epoch [7/30], Loss: 0.7340, Accuracy: 83.56%
Epoch [8/30], Loss: 0.7345, Accuracy: 83.67%
Epoch [9/30], Loss: 0.7665, Accuracy: 81.61%
Epoch [10/30], Loss: 0.7408, Accuracy: 82.99%
Epoch [11/30], Loss: 0.7322, Accuracy: 83.44%
Epoch [12/30], Loss: 0.7248, Accuracy: 83.94%
Epoch [13/30], Loss: 0.7261, Accuracy: 83.93%
Epoch [14/30], Loss: 0.7169, Accuracy: 84.37%
Epoch [15/30], Loss: 0.7142, Accuracy: 84.74%
Epoch [16/30], Loss: 0.7066, Accuracy: 85.09%
Epoch [17/30], Loss: 0.7084, Accuracy: 85.20%
Epoch [18/30], Loss: 0.6935, Accuracy: 85.58%
Epoch [19/30], Loss: 0.6954, Accuracy: 85.60%
Epoch [20/30], Loss: 0.7217, Accuracy: 83.93%
Epoch [21/30], Loss: 0.7306, Accuracy: 83.51%
Epoch [22/30], Loss: 0.6972, Accuracy: 85.4